# Training Model for Binary Classification Problem with Bias Mitigators 

In [1]:
import sys
sys.path.append('../../')

In [2]:
from sklearn.model_selection import train_test_split
from holisticai.pipeline import Pipeline
from holisticai.datasets import load_adult
from holisticai.bias.metrics import classification_bias_metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
np.random.seed(0)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Preprocessing

dataset = load_adult()

df = pd.concat([dataset["data"], dataset["target"]], axis=1)
protected_variables = ["sex", "race"]
output_variable = ["class"]

favorable_label = 1
unfavorable_label = 0

y = df[output_variable].replace(
    {">50K": favorable_label, "<=50K": unfavorable_label}
)
x = pd.get_dummies(df.drop(protected_variables + output_variable, axis=1))
group = ["sex"]
group_a = df[group] == "Female"
group_b = df[group] == "Male"
data = [x, y, group_a, group_b]

dataset = train_test_split(*data, test_size=0.2, shuffle=True)
train_data = dataset[::2]
test_data = dataset[1::2]


In [3]:
import pathlib,os
os.chdir(pathlib.Path(os.path.abspath('')).parent.resolve())

from holisticai.datasets import load_adult
dataset = load_adult()

In [4]:
import pandas as pd

dataset = load_adult()
df = pd.concat([dataset['data'], dataset['target']], axis=1)
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18.0,NaN,103497.0,Some-college,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
48838,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
48839,58.0,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
48840,22.0,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K


## Baseline

In [5]:
    :caption: Tutorials , favorable_label, unfavorable_label = load_preprocessed_adult()
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression()),
    ])

X, y, group_a, group_b = train_data
pipeline.fit(X, y)

X, y, group_a, group_b = test_data
y_pred = pipeline.predict(X)
df_baseline = classification_bias_metrics(group_b.to_numpy().ravel(), 
                            group_a.to_numpy().ravel(), 
                            y_pred.ravel(), 
                            y.to_numpy().ravel())

## Utils

In [6]:
def fit_and_evaluate_pipeline(pipeline, data_cls=None):
    
    if data_cls is None:
        train_data , test_data , favorable_label , unfavorable_label = load_preprocessed_adult()
    else:
        df = data_cls.load_preprocessed_adult_df()
        train_data , test_data , favorable_label , unfavorable_label = data_cls.custom_preprocessing(df)
    
    X, y, group_a, group_b = train_data
    fit_params = {
        'bm__group_a': group_a,
        'bm__group_b': group_b,
        'bm__favorable_label': favorable_label,
        'bm__unfavorable_label': unfavorable_label,
    }
    pipeline.fit(X, y, **fit_params)

    X, y, group_a, group_b = test_data
    predict_params = {
        'bm__group_a': group_a,
        'bm__group_b': group_b,
    }
    y_pred = pipeline.predict(X, **predict_params)
    df = classification_bias_metrics(group_b.to_numpy().ravel(), 
                                group_a.to_numpy().ravel(), 
                                y_pred.ravel(), 
                                y.to_numpy().ravel())
    return df

def format_result_colum(name,config):
    return config['result'].rename(columns={'Value':name}).iloc[:,0]

def show_result_table(configurations, df_baseline):
    table = pd.concat([df_baseline.iloc[:,0]] + [format_result_colum(name,config) 
            for name,config in configurations.items()] + [df_baseline.iloc[:,1]],axis=1)
    return table.rename(columns={'Value':'Baseline'})

## Pre-processing

In [7]:
from collections import defaultdict
configurations = defaultdict(dict)

from holisticai.bias.mitigation.preprocessing import Reweighing
configurations['Reweighing']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_preprocessing', Reweighing()),
    ('classifier', LogisticRegression()),
    ])

from holisticai.bias.mitigation.preprocessing import DisparateImpactRemover
configurations['Disparate Impact Remover']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_preprocessing', DisparateImpactRemover()),
    ('classifier', LogisticRegression()),
    ])

from holisticai.bias.mitigation.preprocessing import LearningFairRepresentation
configurations['Learning Fair Representation']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_preprocessing', LearningFairRepresentation(k=10)),
    ('classifier', LogisticRegression()),
    ])

from holisticai.bias.mitigation.preprocessing import OptimPreproc
data_cls = Dclass()
optim_options = {
    "distortion_fun": data_cls.get_distortion_adult,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
}
configurations['Optim Preproc']['pipeline'] = Pipeline(steps=[
        ('bm_preprocessing', OptimPreproc(optim_options=optim_options)),
        ('scaler', StandardScaler()),
        ('estimator', LogisticRegression())
    ])

In [8]:
for config_name,config in configurations.items():
    if config_name.startswith('Optim'):
        config['result'] = fit_and_evaluate_pipeline(config['pipeline'], data_cls)
    else:
        config['result'] = fit_and_evaluate_pipeline(config['pipeline'])
show_result_table(configurations, df_baseline)

KeyError: 'Age'

## Post Processing

In [ ]:
from collections import defaultdict
configurations = defaultdict(dict)

from holisticai.bias.mitigation.postprocessing import EqualizedOdds
configurations['Equalized Odds']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression()),
    ('bm_postprocessing', EqualizedOdds()),
    ])

from holisticai.bias.mitigation.postprocessing import CalibratedEqualizedOdds
configurations['Calibrated Equalized Odds']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression()),
    ('bm_postprocessing', CalibratedEqualizedOdds()),
    ])

from holisticai.bias.mitigation.postprocessing import RejectOptionClassification
configurations['Reject Option Classification']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression()),
    ('bm_postprocessing', RejectOptionClassification()),
    ])

### Run Configurations

In [ ]:
for config_name,config in configurations.items():
    config['result'] = fit_and_evaluate_pipeline(config['pipeline'])
show_result_table(configurations, df_baseline)

{'class_thresh': 0.21787878787878787, 'roc_margin': 0.12894867037724178, 'balanced_accurracy': 0.7982554875982124, 'fair_metric': 0.04938045535005803}


,Baseline,Equalized Odds,Calibrated Equalized Odds,Reject Option Classification,Reference
Metric,,,,,
Statistical Parity,0.142297,0.080455,0.123103,0.026026,0
Disparate Impact,2.496410,1.561524,2.282961,1.088769,1
Four Fifths Rule,0.400575,0.640400,0.438028,0.918469,1
Cohen D,0.367303,0.203408,0.325580,0.056242,0
Equality of Opportunity Difference,0.137864,0.057090,-0.083333,-0.191182,0
False Positive Rate Difference,0.046398,-0.003797,0.061371,-0.061708,0
Average Odds Difference,0.092131,0.026646,-0.010981,-0.126445,0
Accuracy Difference,-0.069446,-0.044860,-0.144092,0.011224,0
Correlation Difference,0.063582,0.135870,-0.162615,0.070575,0


## Inprocessing

In [ ]:
from collections import defaultdict
configurations = defaultdict(dict)

from holisticai.bias.mitigation.inprocessing import GridSearchReduction

model = LogisticRegression()
inprocessing_model = GridSearchReduction(constraints="DemographicParity", grid_size=20).transform_estimator(model)

configurations['GridSearch Reduction']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_inprocessing', inprocessing_model),
    ])

from holisticai.bias.mitigation.inprocessing import ExponentiatedGradientReduction

model = LogisticRegression()
inprocessing_model = ExponentiatedGradientReduction(constraints="DemographicParity").transform_estimator(model)

configurations['ExponentiatedGradient Reduction']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_inprocessing', inprocessing_model),
    ])
 
from holisticai.bias.mitigation.inprocessing import AdversarialDebiasing

inprocessing_model = AdversarialDebiasing(features_dim=X.shape[1], epochs=50, batch_size=64, hidden_size=64, adversary_loss_weight=0.1, verbose=1, 
                                          use_debias=True).transform_estimator()

configurations['Adversarial Debiasing']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_inprocessing', inprocessing_model),
    ])

from holisticai.bias.mitigation.inprocessing import MetaFairClassifier

inprocessing_model = MetaFairClassifier(tau=0.7, type='fdr').transform_estimator()

configurations['Meta Fair Classifier']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_inprocessing', inprocessing_model),
    ])

from holisticai.bias.mitigation.inprocessing import PrejudiceRemover

inprocessing_model = PrejudiceRemover().transform_estimator()

configurations['Prejudice Remover']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_inprocessing', inprocessing_model),
    ])

from holisticai.bias.mitigation.inprocessing import GerryFairClassifier

inprocessing_model = GerryFairClassifier().transform_estimator()

configurations['Gerry Fair Classifier']['pipeline'] = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_inprocessing', inprocessing_model),
    ])

### Run Configurations

In [ ]:
for config_name,config in configurations.items():
    config['result'] = fit_and_evaluate_pipeline(config['pipeline'])
show_result_table(configurations, df_baseline)

[49,   125] loss: 0.314                 adv_loss: 0.619:  98%|█████████▊| 49/50 [00:28<00:00,  1.74it/s]


,Baseline,GridSearch Reduction,ExponentiatedGradient Reduction,Adversarial Debiasing,Meta Fair Classifier,Prejudice Remover,Gerry Fair Classifier,Reference
Metric,,,,,,,,
Statistical Parity,0.142297,0.001574,-0.009969,0.085059,0.166084,0.096106,0.137614,0
Disparate Impact,2.496410,1.009690,0.936737,1.680473,1.254612,1.878120,2.786232,1
Four Fifths Rule,0.400575,0.990403,0.936737,0.595070,0.797059,0.532447,0.358908,1
Cohen D,0.367303,0.004256,-0.027846,0.221386,0.397105,0.255620,0.371841,0
Equality of Opportunity Difference,0.137864,-0.294304,-0.260343,-0.193632,0.013227,-0.042496,0.049297,0
False Positive Rate Difference,0.046398,-0.035462,-0.059264,0.023876,0.138874,0.011586,0.060136,0
Average Odds Difference,0.092131,-0.164883,-0.159803,-0.084878,0.076051,-0.015455,0.054717,0
Accuracy Difference,-0.069446,-0.087594,-0.066764,-0.114727,0.007264,-0.096846,-0.118868,0
Correlation Difference,0.063582,-0.090189,-0.002041,-0.139514,0.037146,0.015484,-0.043093,0
